# python Python垃圾回收机制Garbage collection(GC)

    Python中的垃圾回收是以引用计数为主，分代收集为辅。引用计数的缺陷是循环引用的问题。
    在Python中，如果一个对象的引用数为0，Python虚拟机就会回收这个对象的内存。
    
    https://www.itcodemonkey.com/article/1197.html
    http://kkpattern.github.io/2015/06/20/python-memory-optimization-zh.html
    https://testerhome.com/topics/16556

In [1]:
class ClassA():
    def __init__(self):
        print('object born,id:%s'%str(hex(id(self))))
    def __del__(self):
        print('object del,id:%s'%str(hex(id(self))))

def f1():
    c1=ClassA()
    del c1

In [3]:
 f1()

object born,id:0x7f77fc024908
object del,id:0x7f77fc024908


# #导致引用计数+1的情况
    1、对象被创建，例如a=23
    2、对象被引用，例如b=a
    3、对象被作为参数，传入到一个函数中，例如func(a)
    4、对象作为一个元素，存储在容器中，例如list1=[a,a]

# #导致引用计数-1的情况
    1、对象的别名被显式销毁，例如del a
    2、对象的别名被赋予新的对象，例如a=24
    3、一个对象离开它的作用域，例如f函数执行完毕时，func函数中的局部变量（全局变量不会）
    4、对象所在的容器被销毁，或从容器中删除对象

##查看一个对象的引用计数
sys.getrefcount(a)可以查看a对象的引用计数，但是比正常计数大1，因为调用函数的时候传入a，这会让a的引用计数+1

In [1]:
#引用计数实验  sys.getrefcount(c) 获取对象在系统中的引用数
import sys

def func(c):
     print('in func function', sys.getrefcount(c) - 1)


print('init', sys.getrefcount(11) - 1)

a = 11
print('after a=11', sys.getrefcount(11) - 1)

b = a
print('after b=1', sys.getrefcount(11) - 1)

func(11)
print('after func(a)', sys.getrefcount(11) - 1)

list1 = [a, 12, 14]
print('after list1=[a,12,14]', sys.getrefcount(11) - 1)

a=12
print('after a=12', sys.getrefcount(11) - 1)

del a
print('after del a', sys.getrefcount(11) - 1)

del b
print('after del b', sys.getrefcount(11) - 1)

del list1
print('after del list1', sys.getrefcount(11) - 1)

init 140
after a=11 141
after b=1 142
in func function 144
after func(a) 142
after list1=[a,12,14] 143
after a=12 142
after del a 142
after del b 141
after del list1 140


# 循环引用导致内存泄露

In [3]:
class ClassA():
    def __init__(self):
        print('object born,id:%s'%str(hex(id(self))))
    def __del__(self):
        print('object del,id:%s'%str(hex(id(self))))
        
def f2():
    c1=ClassA()
    c2=ClassA()
    c1.t=c2
    c2.t=c1
    del c1
    del c2
f2()

object born,id:0x7febe9601588
object born,id:0x7febe9601630


    创建了c1，c2后，0x7febe9601588（c1对应的内存，记为内存1）,0x7febe9601630（c2对应的内存，记为内存2）这两块内存的引用计数都是1.
    执行c1.t=c2和c2.t=c1后，这两块内存的引用计数变成2.

    在del c1后，内存1的对象的引用计数变为1，由于不是为0，所以内存1的对象不会被销毁，所以内存2的对象的引用数依然是2，在del c2后，同理，内存1的对象，内存2的对象的引用数都是1。

    虽然它们两个的对象都是可以被销毁的，但是由于循环引用，导致垃圾回收器都不会回收它们，所以就会导致内存泄露。已经没有外部引用，但内存仍然存在。

# 垃圾回收

In [9]:
import time
import gc


class ClassA():
    def __init__(self):
        print('object born,id:%s'%str(hex(id(self))))
    def __del__(self):
        print('object del,id:%s'%str(hex(id(self))))
        
def f3():
    c1=ClassA()
    c2=ClassA()
    c1.t=c2
    c2.t=c1
    del c1
    del c2
    gc.garbage#print("垃圾回收前：",gc.garbage)
    print("垃圾回收个数：",gc.collect()) #显式执行垃圾回收
    gc.garbage#print("垃圾回收后：",gc.garbage)
    time.sleep(1)
if __name__ == '__main__':
    gc.set_debug(1) #设置gc模块的日志gc.DEBUG_LEAK
    f3() #相比于f2, f3 调用 gc.collect() 解决了循环引用问题
    

object born,id:0x7ff59029f518
object born,id:0x7ff59029f400
object del,id:0x7ff59029f518
object del,id:0x7ff59029f400
垃圾回收个数： 11


gc: collecting generation 2...
gc: objects in each generation: 445 0 50579
gc: done, 11 unreachable, 0 uncollectable, 0.0135s elapsed


垃圾回收后的对象会放在gc.garbage列表里面
gc.collect()会返回外部引用不可达的对象数目，11为两个对象以及它们在jupyter中额外对应的dict

有三种情况会触发垃圾回收：
    1.调用gc.collect(),
    2.当gc模块的计数器达到阀值的时候（自动垃圾回收）。
    3.程序退出的时候

# gc模块常用功能解析

常用函数：
 
     gc.set_debug(flags)
    设置gc的debug日志，一般设置为gc.DEBUG_LEAK
    
    gc.collect([generation])
    显式进行垃圾回收，可以输入参数，0代表只检查第一代的对象，1代表检查一，二代的对象，2代表检查一，二，三代的对象，如果不传参数，执行一个full collection，也就是等于传2。
    返回不可达（unreachable objects）对象的数目
    
    gc.set_threshold(threshold0[, threshold1[, threshold2])
    设置自动执行垃圾回收的频率。
    
    gc.get_count()
    获取当前自动执行垃圾回收的计数器，返回一个长度为3的列表

# gc模块的自动垃圾回收触发机制
必须要import gc模块，垃圾回收机制python默认开启的，gc.disable() 可关闭垃圾回收机制，当程序完成时，垃圾最后仍被回收
这个机制的主要作用就是发现并处理不可达的垃圾对象。

    在Python中，采用分代收集的方法。把对象分为三代，一开始，对象在创建的时候，放在一代中，如果在一次一代的垃圾检查中，若对象存活下来，就会被放到二代中，同理在一次二代的垃圾检查中，该对象存活下来，就会被放到三代中。

    gc模块里面会有一个长度为3的列表的计数器，可以通过gc.get_count()获取。

    例如(488,3,0)，其中488是指距离上一次一代垃圾检查，Python分配内存的数目减去释放内存的数目，注意是内存分配，而不是引用计数的增加。3是指距离上一次二代垃圾检查，一代垃圾检查的次数，同理，0是指距离上一次三代垃圾检查，二代垃圾检查的次数。

In [12]:
import time
import gc

class ClassA():
    def __init__(self):
        print('object born,id:%s'%str(hex(id(self))))
    def __del__(self):
        print('object del,id:%s'%str(hex(id(self))))


print(gc.get_count()) 
a = ClassA()
print(gc.get_count()) 
del a
print(gc.get_count()) 

(23, 0, 0)
object born,id:0x7f9c6855ccf8
(48, 0, 0)
object del,id:0x7f9c6855ccf8
(34, 0, 0)


gc: collectable <type 0x10cf4e8>
gc: collectable <tuple 0x7f9c684f6688>
gc: collectable <dict 0x7f9c6849ef78>
gc: collectable <function 0x7f9c6848bd90>
gc: collectable <function 0x7f9c6848be18>
gc: collectable <getset_descriptor 0x7f9c68427480>
gc: collectable <getset_descriptor 0x7f9c684274c8>


### gc模快有一个自动垃圾回收的阀值，即通过gc.get_threshold函数获取到的长度为3的元组，例如(700,10,10) 每一次计数器的增加，gc模块就会检查增加后的计数是否达到阀值的数目，如果是，就会执行对应的代数的垃圾检查，然后重置计数器

#### 假设阀值是(700,10,10)：
    当计数器从(699,3,0)增加到(700,3,0)，gc模块就会执行gc.collect(0),即检查一代对象的垃圾，并重置计数器为(0,4,0)
    当计数器从(699,9,0)增加到(700,9,0)，gc模块就会执行gc.collect(1),即检查一、二代对象的垃圾，并重置计数器为(0,0,1)
    当计数器从(699,9,9)增加到(700,9,9)，gc模块就会执行gc.collect(2),即检查一、二、三代对象的垃圾，并重置计数器为(0,0,0)

        被分配对象的计数值与被释放对象的计数值之间的差异在逐渐增长。一旦这个差异累计超过某个阈值，
    则Python的收集机制就启动了,将内存进行代数转移（提升）
    
    垃圾回收器会更频繁的处理新对象。一个新的对象即是你的程序刚刚创建的，而一个来的对象则是经过了几个时间周期之后仍然存在的对象。Python会在当一个对象从一代移动到二代，或是从二代移动到三代的过程中提升(promote)这个对象。
    为什么要这么做？这种算法的根源来自于弱代假说(weak generational hypothesis)。这个假说由两个观点构成：首先是年亲的对象通常死得也快，而老对象则很有可能存活更长的时间。

# python中避免出现内存泄漏的方法

    1、项目中避免循环引用
    
    2、引入gc模块，启动gc模块的自动清理循环引用的对象机制
    
    3、由于分代收集，所以把需要长期使用的变量集中管理，并尽快移到二代以后，减少GC检查时的消耗
    
    4、gc模块唯一处理不了的是循环引用的类都有__del__方法，因为gc模块不知道要先调用哪个方法，所以项目中要避免定义__del__方法，如果一定要使用该方法，同时导致了循环引用，需要代码显式调用gc.garbage里面的对象的__del__来打破僵局

# python自动垃圾回收机制是默认开启的，有时会比较耗时
# 可以进行必要的性能调整

#### （1） 在这种情况下，首先要使用 gc 模块的 set_debug() 来查找原因
#### import gc
#### gc.set_debug(gc.DEBUG_STATS)
#### gc.collect()
####  #gc: collecting generation 2...
####  #gc: objects in each generation: 10 0 13607
####  #gc: done, 0.0087s elapsed.

一旦用 set_debug() 设定了 gc.DEBUG_STATS 标志，那么每次进行循环引用垃圾回收，
就都会输出以下信息。
1. GC 对象的代
2. 各代内对象的数量
3. 循环引用垃圾回收所花费的时间

####  （2）使用 gc.collect() ，就能在应用程序运行过程中的任意时刻执行循环引用垃圾回收了。也就是说，这样一来就可以在应用程序空闲或者等待执行的期间执行 GC 了。

####  （3）用 gc.disable() 试试吧，这也是一种手段。一旦调用 gc.disable()，循环引用垃圾回收就停止运作了。也就是说，循环引用的垃圾对象群一直不会得到释放。然而从应用程序整体的角度来看，如果循环引用的对象的大小可以忽视，那么这个方法也不失为一个好方法。这就需要我们自己来权衡了。由于一次垃圾回收比较耗时，可以调高垃圾回收阈值，减少垃圾回收频率

####  (4)避免循环引用或使用弱引用（weakref库）

# 总的来说，python的内存管理机制是以引用计数为主的，引用计数会导致循环引用

# 的对象内存无法清除，从而python引入了基于分代回收的自动垃圾回收机制（比较

# 耗时，默认），来清除存在循环引用的对象内存，这个自动垃圾回收机制（通过

# gc.set_threshold来调整回收周期）可以手动回收（gc.collect()语句）


# python3.4之后没有下列问题了

# .........................................分隔线.......................................................

# python默认的计数引用和垃圾回收内存管理，还是会导致内存泄露！

#### 循环引用中的对象定义了__del__函数，这个在《程序员必知的Python陷阱与缺陷列表》一文中有详细介绍，简而言之，如果定义了__del__函数，那么在循环引用中Python解释器无法判断析构对象的顺序，因此就不会处理（python3.4)   python 3.6已经不存在这个问题了

In [10]:
import gc

class T2(object):
	def __del__(self):
		pass

def test_uncollectable():
	a = T2()
	b = T2()
	a.child = b
	b.parent = a

gc.collect()
gc.set_debug(gc.DEBUG_UNCOLLECTABLE | gc.DEBUG_LEAK)
test_uncollectable()
gc.collect()

gc: collectable <T2 0x7f9c68509b70>
gc: collectable <T2 0x7f9c68509fd0>
gc: collectable <dict 0x7f9c6849a2d0>
gc: collectable <dict 0x7f9c6849ac18>


4